# Import statements
Following packages are necessary:

In [1]:
import sys
sys.path.append("D:\Julian\workspace\endolas\packages")

import random
from endolas import utils
from endolas import ccall

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from endolas import UNet
from endolas import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from endolas import LASTENSequence
from matplotlib import pyplot as plt

Segmentation Models: using `tf.keras` framework.


# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.get_visible_devices('GPU'))
print(tf.config.get_visible_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# 3.4) Supervised Segmentation without augmentation and Tensorboard
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN2 data set.

# Data
Import training images

In [4]:
tf.keras.backend.clear_session()

#path_train = r'D:\Julian\data\LASTEN2\train'
#path_validation = r'D:\Julian\data\LASTEN2\validation'
#path_test = r'D:\Julian\data\LASTEN2\test'

path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/train'
path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/validation'
path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/test'

width = 384 #768
height = 384 #768

# Seeding
Seeds are set to ensure reproducible training.

In [5]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [6]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 384, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 384, 384, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 384, 384, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 384, 384, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [7]:
train_gen = LASTENSequence(path_train,
                           batch_size=1,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=False,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=1,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=2,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

In [ ]:
train_gen[0]

> /home/julian/Documents/Studium/MT-Masterarbeit/Workspace/endolas/packages/endolas/lastengen.py(306)_preprocess_image()
    304         set_trace()
    305 
--> 306         return image
    307 
    308     def _run_augmentation(self, image, mask):

ipdb> image
array([[[2.],
        [2.],
        [2.],
        ...,
        [2.],
        [2.],
        [1.]],

       [[1.],
        [1.],
        [2.],
        ...,
        [2.],
        [2.],
        [2.]],

       [[1.],
        [2.],
        [2.],
        ...,
        [2.],
        [2.],
        [2.]],

       ...,

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [0.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [2.],
        [2.],
        [0.]],

       [[2.],
        [2.],
        [1.],
        ...,
        [1.],
        [0.],
        [0.]]], dtype=float32)
ipdb> image.shape
(384, 384, 1)
ipdb> image.max
<built-in method max of numpy.ndarray object at 0x7f1458200ad0>
ipd

# Training Preparation
Prepare the settings for training the model.

In [ ]:
aim_path= "3_4_segmentation"

save_path = "../results/" + aim_path

logger = CSVLogger(save_path + "/log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5", period=10)
checker_best = ModelCheckpoint(save_path + "/best_weights.hdf5", save_best_only=True)
timelogger = ccall.TimeHistory(save_path)
tensorboard_callback = tf.keras.callbacks.TensorBoard("../tf_logs/"+aim_path, histogram_freq=1)

callbacks = [logger, checker, checker_best, timelogger, tensorboard_callback]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [ ]:
model.fit(train_gen,
                    epochs=2,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    #use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

# Evaluation

In [ ]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

In [ ]:
test_gen = LASTENSequence(path_train,
                          batch_size=1,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)